<a href="https://colab.research.google.com/github/edzix-1/NLP-Project/blob/main/Polish_Croatian_translations_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code responsible for downloading and unpacking the data

In [2]:
import urllib.request

url = "https://object.pouta.csc.fi/OPUS-MultiParaCrawl/v7.1/moses/hr-pl.txt.zip"
output_path = "opus_pl_hr.zip"

urllib.request.urlretrieve(url, output_path)
print("Download complete")


Download complete


In [3]:
!unzip -o opus_pl_hr.zip

Archive:  opus_pl_hr.zip
  inflating: README                  
  inflating: LICENSE                 
  inflating: MultiParaCrawl.hr-pl.hr  
  inflating: MultiParaCrawl.hr-pl.pl  
  inflating: MultiParaCrawl.hr-pl.xml  


In [4]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu

with open("MultiParaCrawl.hr-pl.hr", "r", encoding="utf-8") as f: hr = f.read().splitlines()
with open("MultiParaCrawl.hr-pl.pl", "r", encoding="utf-8") as f: pl = f.read().splitlines()

df = pd.DataFrame({"hr": hr, "pl": pl})
n = 300

Multilingual model and tokenizer declaration. Creating the translate function for the Multilingual model.

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

def translate_multi(sentences, src_lang, tgt_lang):
    output = []
    tokenizer.src_lang = src_lang
    for sentence in sentences:
      encoded = tokenizer(sentence, return_tensors="pt")

      generated_tokens = model.generate(
          **encoded,
          forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang]
      )
      output.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

    return output

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Back-and-forth translations and evaluations for the Multilingual model

In [5]:
hr_to_pl = translate_multi(hr[:n], "hr_HR", "pl_PL")

In [6]:
total_score = 0
for i in range(n):
  bleu = sentence_bleu([pl[:n][i].split(' ')], hr_to_pl[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.05029973020798613


/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [7]:
pl_back = translate_multi(hr_to_pl, "pl_PL", "hr_HR")
total_score = 0
for i in range(n):
  bleu = sentence_bleu([hr[:n][i].split(' ')], pl_back[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.030494613320098042


In [6]:
pl_to_hr = translate_multi(pl[:n], "pl_PL", "hr_HR")
total_score = 0
for i in range(n):
  bleu = sentence_bleu([pl[:n][i].split(' ')], pl_to_hr[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.022715905821878768


/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [7]:
hr_back = translate_multi(pl_to_hr, "hr_HR", "pl_PL")
total_score = 0
for i in range(n):
  bleu = sentence_bleu([hr[:n][i].split(' ')], hr_back[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.00667026818647875


Translations through English

In [8]:
ht_to_en = translate_multi(hr[:n], "hr_HR", "en_XX")
en_to_pl = translate_multi(ht_to_en, "en_XX", "pl_PL")
total_score = 0
for i in range(n):
  bleu = sentence_bleu([pl[:n][i].split(' ')], en_to_pl[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.08704530202534032


Slavic model and tokenizer declaration. Creating the translate function for the Slavic model.

In [9]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-sla-sla'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
def translate(text, trg_lang):
    text = ">>" + trg_lang + "<< " + text
    translated = model.generate(**tokenizer(text, return_tensors='pt', padding=True))

    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    return translated_text

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/877k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/867k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:176: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


...

pytorch_model.bin:   0%|          | 0.00/257M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/258 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/257M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Back-and-forth translations and evaluations for the Slavic model

In [10]:
hr_to_pl2 = []
for i in range(n):
  hr_to_pl2.append(translate(hr[i], "pol"))

total_score = 0
for i in range(n):
  bleu = sentence_bleu([pl[:n][i].split(' ')], hr_to_pl2[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.10783676185330641


In [11]:
pl_to_hr2 = []
for i in range(n):
  pl_to_hr2.append(translate(pl[i], "hrv"))

total_score = 0
for i in range(n):
  bleu = sentence_bleu([hr[:n][i].split(' ')], pl_to_hr2[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.11768981773811153


In [12]:
hr_back2 = []
for i in range(n):
  hr_back2.append(translate(pl_to_hr2[i], "pol"))

total_score = 0
for i in range(n):
  bleu = sentence_bleu([pl[:n][i].split(' ')], hr_back2[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.18825458515966959


In [13]:
pl_back2 = []
for i in range(n):
  pl_back2.append(translate(hr_to_pl2[i], "hrv"))

total_score = 0
for i in range(n):
  bleu = sentence_bleu([hr[:n][i].split(' ')], pl_back2[i].split(' '))
  total_score += bleu
print("BLEU score: ", total_score/n)

BLEU score:  0.19537810988642682


Text results

In [ ]:
hr_to_pl

In [ ]:
pl_back

In [19]:
pl_to_hr

['Toliko sam u svom životu vidjela djevojke, ne može se ne usporediti.',
 'Za većinu njih, smoke in their own room is not enough, unlike going out and shouting so that everybody can see.',
 'Kako organizirati 8 March u uredu',
 'Our mission - We are traditional house and related middle family in order to maintain and develop fruit baking carrier and promote culture of pleasure to be independent.',
 'Tanim privates i malim, prijaznim dorms u centru Krakowa Brak recenze',
 'Błędna je ideja da se razvoj govora počinje od trenutka prve riječi.',
 'No, if you are one of baldness or hair loss around a person, the factor potentially isn’t actually your biggest concern today.',
 'Wojna tek počinje...',
 "Turtle's fall leads heirs to a review of Westing's will to get rid of his true significance.",
 'Średnja ocena naših bivših studenata, na pitanje o njihovom cijelom stayu u školi u Frankfurtu',
 'U this genre sceneria is often a fallen society, usually occurring after a great war or another fr

In [18]:
hr_back

["I've seen so many girls in my life, it's niesamowite.",
 'Dla większości, smoke in their own room is not enough, unlike going out and shouting so that everybody can see.',
 'Jake to zorganizować 8 March w biurze?',
 'Our mission — We are traditional house and related middle family in order to maintain and develop fruit baking carrier and promote culture of pleasure to be independent.',
 'Tanie privates i mały, przyjazny dorms w centrum Krakowa Brak recenzji',
 'Błędne jest to myśl, że rozwój języka zaczyna się od momentu pierwszego słowa.',
 "Ale, if you are one of baldness or hair loss around a person, the factor potentially isn't actually your biggest concern today.",
 'Wojna tylko zaczyna...',
 "Turtle's fall leads heirs to a review of Westing's will to get rid of his true significance.",
 'Średnia ocena naszych byłych studentów, na pytanie o ich cały pobyt w szkole w Frankfurt.',
 'In this genre sceneria is often a fallen society, usually occurring after a great war or another fr

In [17]:
en_to_pl

['Widziałem tak wiele dziewczyn w moim życiu, nie można ich nie porównać.',
 'Dla większości z nich, to, że są w swoich pokojach w prywatności, nie wystarczy, ale wystarczy, aby wyjść na zewnątrz i uprawiać seks przed każdym, kogo mogą zobaczyć.',
 'Jak planujesz 8 marca w biurze?',
 'Nasza misja – jesteśmy tradycyjnymi breadwinnerami i breadwinnerami rodziny średniej z celem utrzymania i rozwoju owoców, żywiącymi i ułatwiającymi kulturę radości, która jest jak najbardziej niezależna.',
 'Olejki płciowe i małe, wygodne akademiki w centrum Krakowa.',
 'Jest to błędne pojęcie, że mówienie rozwija się od momentu, gdy mówi się pierwsze słowo.',
 'Jednak, jeśli jesteś jednym z stratników włosów lub rzęsów, powód prawdopodobnie nie jest twoim największym zaniepokojeniem dzisiaj.',
 'Wojna dopiero się rozpoczęła...',
 'Upadek ruchu Kornjac prowadzi spadkobierców do przeglądu woli Westinga do wyjaśnienia jego prawdziwego znaczenia.',
 'Średnia ocena naszych byłych uczniów, kiedy zapytaliśmy ic

In [16]:
pl[:n]

['Jestem tak bardzo w moim życiu widziałem dziewczyny, nie można nie być porównywane. ',
 'Dla większości z nich, dymanie we własnym pokoju to za mało, w przeciwieństwie do wyjścia na zewnątrz i bzykanie się taki, żeby wszyscy widzieli. ',
 'Jak zorganizować 8 marca w biurze ',
 'Nasza misja - Jesteśmy tradycyjnym domu i związane z nimi średnie rodzina w celu utrzymania i rozwoju owoców pieczenie przewoźnika i promotorem kultury przyjemności, aby były niezależne. ',
 'Tanie privates i mały, przytulny dorms w centrum Krakowa Brak recenzji ',
 'Błędna jest opinia, że rozwój mowy rozpoczyna się od momentu pierwszych słów. ',
 'Jednak jeśli jesteś jednym z bezwłosy lub utrata włosów wokół osoby, czynnik potencjalnie nie jest faktycznie największym zmartwieniem swoją dzisiaj. ',
 'Wojna dopiero się zaczyna... ',
 'Upadek Turtle prowadzi spadkobierców w przeglądzie woli Westinga, aby pozbyć się prawdziwego znaczenia. ',
 'Średnia ocena naszych byłych uczniów, na pytanie o ich cały pobyt w sz

In [15]:
hr_to_pl2


['W życiu widziałem wszystkie dziewczyny. Nie można się porównać.',
 'Dla większości z nich, pieprzenie się w prywatności swojego pokoju nie jest wystarczające, ale podnieca ich wyjazd na zewnątrz i seks przed każdym, kto ich widzi.',
 'Jak organizować 8 marca w biurze',
 'Nasza misja - Jesteśmy tradycyjną rodziną domową i śledzącą środkową w celu utrzymania i rozwoju owoców spala nosiciel i promotor kultury przyjemności, aby być jak najwięcej niezależną.',
 'Tanie genitalie i małe, przyjemne dorms w centrum Krakow',
 'Niewłaściwe jest opinia, że rozwój mowy zaczyna się od momentu pierwszych słów.',
 'Mimo to, jeśli jesteś jednym z włosów lub straty ludzi, to powód potencjalnie nie jest twoim największym problemem dzisiaj.',
 'Wojna dopiero się zaczęła.',
 'Działanie żółwia prowadzi spadkobierców do przeglądu woli Westinga, by wyjaśnić swoje prawdziwe znaczenie.',
 'Średnia ocena naszych byłych uczniów, kiedy zapytaliśmy ich o ich wspólne pobyt w szkole w Frankfurt',
 'W tym gatunku, ś

In [14]:
hr_back2

['Widziałem tyle dziewczyn w moim życiu, nie można porównać.',
 'Dla większości z nich dym w swoim pokoju nie wystarczy, w przeciwieństwie do wyjścia i pieprzenia, aby wszyscy widzieli.',
 'Jak organizować 8 marca w biurze',
 'Nasza misja - Jesteśmy tradycyjnym domem i jest z nimi związana średnia rodzina do utrzymania i rozwijania owoców pieczenia przewoźników i promotora kultury satysfakcji, aby być niezależnym.',
 'Tanie privates i mały, miły dorms w centrum Krakow Nie ma recenzji',
 'Niewłaściwe stwierdzenie, że rozwój mowy rozpoczął się od początku pierwszych słów.',
 'Ale jeśli jesteś jednym z bez włosów lub utratą włosów wokół osoby, to to nie jest potencjalnie największym zaniepokojeniem w dzisiejszych czasach.',
 'Wojna dopiero się zaczęła.',
 'Upadek żółwia prowadzi spadkobierców do przeglądu woli Westing, by pozbyć się prawdziwego znaczenia.',
 'Średnia ocena naszych byłych uczniów na temat ich pobytu w szkole w Frankfurt',
 'W takich scenariuszach często spadło społeczeństw